## Создание сессии Spark

In [ ]:
import findspark
findspark.init()
findspark.find()

In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder\
    .master("local[1]")\
    .appName("task_47")\
    .config("spark.executor.memory", "10g")\
    .config("spark.executor.cores", 5)\
    .config("spark.dynamicAllocation.enabled", "true")\
    .config("spark.dynamicAllocation.maxExecutors", 5)\
    .config("spark.shuffle.service.enabled", "true")\
.getOrCreate()

## Задание 4.8.1

In [ ]:
#исходные данные
row_data = ((1, 1562007679), (1, 1562007710), (1, 1562007720), (1, 1562007750),\
            (2, 1564682430), (2, 1564682450), (2, 1564682480))

columns= ["id", "time_stamp"]

df = spark.createDataFrame(data = row_data, schema = columns)

In [ ]:
df.printSchema()

In [ ]:
df.show(truncate=False)

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import min, max

windowSpec  = Window.partitionBy("id")

df = df.withColumn("minn", min("time_stamp").over(windowSpec))\
       .withColumn("maxx", max("time_stamp").over(windowSpec))

In [ ]:
temp_table_name = 'temp'
df.createOrReplaceTempView(temp_table_name)

sql = '''
    select distinct id, maxx - minn 
    from temp
'''

df = spark.sql(sql).withColumnRenamed('(maxx - minn)', 'Длительность сессии, сек')
df.show(truncate=False)

In [ ]:
spark.stop